In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

import db_queries as db
import vivarium_helpers.id_helper as idh

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
import model_validation.vivarium_raw_output as vro

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Tue Aug  3 19:04:19 PDT 2021


In [4]:
%load_ext autoreload
%autoreload 2

In [79]:
import re

# Define directories

In [5]:
# sim results path:
# /ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data

project_results_dir = '/ihme/costeffectiveness/results/vivarium_ciff_sam'
model_name = 'v2.3_wasting_birth_prevalence'
model_timestamp = '2021_07_26_17_14_31'

model_count_data_dir = f'{project_results_dir}/{model_name}/ciff_sam/{model_timestamp}/count_data/'

project_vv_directory_name = 'ciff_malnutrition/verification_and_validation'
username = 'ndbs'

output_dir = f'/ihme/homes/{username}/vivarium_results/{project_vv_directory_name}/{model_name}'

share_output_dir = f'/share/scratch/users/ndbs/vivarium_results/{project_vv_directory_name}/{model_name}'
j_output_dir = f'/home/j/Project/simulation_science/{project_vv_directory_name}/{model_name}'

In [93]:
data = vto.VivariumTransformedOutput.from_directory(model_count_data_dir)
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [94]:
data.deaths

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [8]:
data['deaths']

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [9]:
data['yak'] = '4'

TypeError: 'VivariumTransformedOutput' object does not support item assignment

In [10]:
data.yak = pd.DataFrame(range(4))

In [11]:
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'yak']

In [12]:
data['yak']

,0
0,0
1,1
2,2
3,3


In [13]:
del data.yak

In [14]:
data.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [15]:
'value' in data.deaths

True

In [16]:
data.deaths.columns

Index(['sex', 'year', 'cause', 'measure', 'input_draw', 'scenario', 'value',
       'wasting_state', 'age'],
      dtype='object')

In [17]:
['value'] in data.deaths

TypeError: unhashable type: 'list'

In [19]:
isinstance(['value'], collections.Hashable)

False

In [20]:
isinstance('value', collections.Hashable)

True

In [77]:
def method1(colnames, df):
    if isinstance(colnames, collections.Hashable):
        if colnames in df: # assume colnames is a single column name in df
            colnames = [colnames]
#         elif isinstance(colnames, str): # Assume colnames is supposed to be a single column name
#             raise KeyError(f"string {colnames} not in the DataFrame")
    elif isinstance(colnames, collections.Iterable): # assume colname is an iterable of column names
        colnames = colnames # Note - this will not catch strings that are bad column names
    else:
        raise ValueError(f"colnames must be a single column name in df or an iterable of column names")
        
def method2(colnames, df):
    # Assume that if colnames is hashable it represents a single column name,
    # and otherwise it must be an iterable of column names
    if isinstance(colnames, collections.Hashable):
        # This line could still raise an 'unhashable type' TypeError if e.g. colnames is a tuple that contains an unhashable type
        if colnames in df: # assume colnames is a single column name in df
            colnames = [colnames]
        else: # Assume colnames is supposed to be a single column name
            raise KeyError(f"Key {colnames=} not in the DataFrame")
    elif not isinstance(colnames, collections.Iterable): # assume colname is an iterable of column names
        raise ValueError(f"{colnames=} must be a single column name in df or an iterable of column names")

In [22]:
a = np.array(range(4))
a

array([0, 1, 2, 3])

In [23]:
isinstance(a, collections.Hashable)

False

In [24]:
isinstance(a, collections.Iterable)

True

In [25]:
class TestCollection(collections.abc.Mapping):
    def __init__(self, table_dict):
        self.__dict__ = dict(table_dict)

    def __getitem__(self, table_name):
        return self.__dict__[table_name]
    
    def __iter__(self):
        return iter(self.__dict__)

# # What happens if we don't implement something we're supposed to? Then we can't instantiate the class.
#     def __len__(self):
#         return len(self.__dict__)

    def to_dict(self):
        return dict(self.__dict__)

    def table_names(self):
        return list(self.keys())

In [26]:
tc = TestCollection(data)

TypeError: Can't instantiate abstract class TestCollection with abstract methods __len__

In [27]:
data2 = vto.VivariumTransformedOutput(data)
data2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [29]:
locations_paths = {
    'Ethiopia': model_count_data_dir,
    'Fake_location': model_count_data_dir,
}
d = vto.load_count_data_by_location(locations_paths, '')
d.keys()

dict_keys(['Ethiopia', 'Fake_location'])

In [31]:
d['Ethiopia'].keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count'])

In [33]:
d2 = vto.VivariumTransformedOutput.from_locations_paths(locations_paths, '')
d2.keys()

KeysView(<model_validation.vivarium_transformed_output.VivariumTransformedOutput object at 0x2b8e6ff46b50>)

In [34]:
d2

In [36]:
d2.table_names()

['ethiopia', 'fake_location']

In [37]:
d2.ethiopia.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [38]:
d2.fake_location.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [43]:
d2.fake_location.wasting_state_person_time

,sex,year,cause,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,67.629021,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,242.324435,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,3526.287474,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,8192.839151,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,4633.670089,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,5516.294319,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,11032.581793,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,63806.663929,2_to_4


In [41]:
d3 = vto.VivariumTransformedOutput(vto.merge_location_count_data(d2))
d3.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [42]:
d3.wasting_state_person_time

,location,sex,year,cause,measure,input_draw,scenario,value,age
0,ethiopia,female,2022,mild_child_wasting,state_person_time,29,baseline,67.629021,early_neonatal
1,ethiopia,female,2022,mild_child_wasting,state_person_time,29,baseline,242.324435,late_neonatal
2,ethiopia,female,2022,mild_child_wasting,state_person_time,29,baseline,3526.287474,1-5_months
3,ethiopia,female,2022,mild_child_wasting,state_person_time,29,baseline,8192.839151,6-11_months
...,...,...,...,...,...,...,...,...,...
2876,fake_location,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,4633.670089,1-5_months
2877,fake_location,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,5516.294319,6-11_months
2878,fake_location,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,11032.581793,12_to_23_months
2879,fake_location,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,63806.663929,2_to_4


In [44]:
test = vto.VivariumTransformedOutput({'yak': 3, 'bad variable': 4})
test.table_names()

['yak', 'bad variable']

In [45]:
test.bad variable

SyntaxError: invalid syntax (3398138285.py, line 1)

In [46]:
test.yak

3

In [47]:
test['bad variable']

4

In [48]:
setattr(test, 'goat', 5)
test.table_names()

['yak', 'bad variable', 'goat']

In [49]:
test.goat

5

In [50]:
setattr(test, 'uh oh', 6)
test.table_names()

['yak', 'bad variable', 'goat', 'uh oh']

In [52]:
test.uh oh

SyntaxError: invalid syntax (3240682433.py, line 1)

In [54]:
d4 = vto.VivariumTransformedOutput.merged_from_locations_paths(locations_paths, '')
d4.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [55]:
d4.deaths

,location,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,Ethiopia,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,Ethiopia,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,Ethiopia,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,Ethiopia,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
17276,Fake_location,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,Fake_location,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,Fake_location,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,Fake_location,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [56]:
import db_queries as db
import vivarium_helpers.id_helper as idh

In [57]:
idh.search_id_table('location', 'ivoire', case=False)

,location_id,location_name,location_type,location_description
174,205,Côte d'Ivoire,admin0,admin0


In [59]:
cote_divoire = idh.search_id_table('location', 'ivoire', case=False)['location_name'].iloc[0]
cote_divoire

"Côte d'Ivoire"

In [60]:
locations_paths2 = dict(locations_paths)
locations_paths2[cote_divoire] = locations_paths['Ethiopia']
locations_paths2

{'Ethiopia': '/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data/',
 'Fake_location': '/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data/',
 "Côte d'Ivoire": '/ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data/'}

In [62]:
test2 = vto.VivariumTransformedOutput.from_locations_paths(locations_paths2, '')
test2.table_names()

['ethiopia', 'fake_location', "côte_d'ivoire"]

In [63]:
test2.côte_d'ivoire

SyntaxError: EOL while scanning string literal (4267570992.py, line 1)

In [95]:
test3 = vto.VivariumTransformedOutput.from_locations_paths(locations_paths2, '')
test3.table_names()

['ethiopia', 'fake_location', 'côte_d_ivoire']

In [96]:
test3.côte_d_ivoire

In [97]:
test3.côte_d_ivoire.deaths

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [68]:
isinstance((1,(2,3)), collections.Hashable)

True

In [69]:
isinstance((1,[2,3]), collections.Hashable)

True

In [70]:
isinstance((1,a), collections.Hashable)

True

In [72]:
(1,a) in test3.côte_d_ivoire.deaths

TypeError: unhashable type: 'numpy.ndarray'

In [73]:
anindex = pd.Index(range(5))
anindex

RangeIndex(start=0, stop=5, step=1)

In [74]:
isinstance(anindex, collections.Hashable)

False

In [75]:
isinstance(pd.Index((1,2,3)), collections.Hashable)

False

In [76]:
pd.Index((1,2,3))

Int64Index([1, 2, 3], dtype='int64')

In [88]:
def make_variable_name(string): return re.sub('\W+|^(?=\d)','_', string)

list(map(make_variable_name, ['123abc', 'ac', "abc'def_g__hi", "4 yak''''yak. yak   grunt", 'a.b.c.d 456 789']))

['_123abc', 'ac', 'abc_def_g__hi', '_4_yak_yak_yak_grunt', 'a_b_c_d_456_789']

In [80]:
# re.sub('\W+|^(?=\d)','_', string)
re.findall('\W+|^(?=\d)', 'abc123')

[]

In [81]:
re.findall('\W+|^(?=\d)', '3abc123')

['']

In [89]:
côte_d_ivoire = cote_divoire # apparently you can have unicode characters in variable names... woo hoo!

In [90]:
côte_d_ivoire

"Côte d'Ivoire"